In [ ]:
import numpy as np
import pandas as pd
from scipy.integrate import solve_ivp
from scipy.stats import quantile
from itertools import combinations

# Load source_ACR module
# Assuming the relevant functions are defined in source_ACR.py
import source_ACR as acr

# Initialization
eps_acr = 0.01
thres_positive = 0.01
num_S = 2
max_order = 2
num_R = 3
source_mat = None
product_mat = None
stoi_mat = None

def MAK(t, x, kappa):
    f = np.prod(np.power(x, source_mat), axis=0)
    r = kappa * f
    dx_vec = np.dot(stoi_mat, r)
    return dx_vec

def MAK_rsc(t, x, kappa):
    f = np.prod(np.power(x, source_mat), axis=0)
    r = kappa * f
    dx_vec = np.dot(stoi_mat, r) / (1 + np.sqrt(np.sum(dx_vec**2)))
    return dx_vec

num_total_C = acr.binomial(max_order + num_S, num_S)
total_complex = acr.make_total_complexes(num_S, max_order)

num_total_R = num_total_C * (num_total_C - 1)
num_total_net = acr.binomial(num_total_R, num_R)

num_repeat_net = num_total_net
num_repeat_par = 5
num_repeat_init = 20
matrix_R_id = np.zeros((num_R, num_repeat_net), dtype=int)
list_acr_id = np.zeros((num_S, num_repeat_net), dtype=int)
list_unbnd_id = np.zeros((num_S, num_repeat_net), dtype=int)

# Beginning of the random network search
rnd_search = False
for iter_network in range(1, num_repeat_net + 1):
    if iter_network % 10 == 1:
        print(f"======= Analyzing network {iter_network:4} =======")

    if rnd_search:
        list_R_id = np.random.choice(range(1, num_total_R + 1), num_R, replace=False)
    else:
        list_R_id = acr.net_id_to_r_id_list(iter_network, num_total_R, num_R)

    list_source_id, list_product_id = zip(*[acr.reaction_to_complex(r_id, num_total_C) for r_id in list_R_id])
    source_mat = total_complex[:, list_source_id]
    product_mat = total_complex[:, list_product_id]
    stoi_mat = product_mat - source_mat

    # Simulation settings
    kappa1 = np.random.uniform(low=0.0, high=10.0, size=num_R)
    x_init = np.random.uniform(low=1.0, high=20.0, size=num_S)
    tspan1 = (0.0, 50.0)

    ub_param = 10 * np.ones(num_R)
    lb_param = np.zeros(num_R)

    ub_init = 20 * np.ones(num_S)
    lb_init = np.ones(num_S)

    list_acr_id_par = np.zeros((num_S, num_repeat_par), dtype=int)
    list_unbnd_id_par = np.zeros((num_S, num_repeat_par), dtype=int)

    for iter_par in range(1, num_repeat_par + 1):
        kappa1 = np.random.uniform(low=lb_param, high=ub_param)
        final_val_mat = np.zeros((num_S, num_repeat_init))
        latter_val_mat = np.zeros((num_S, num_repeat_init))

        for iter_init in range(1, num_repeat_init + 1):
            x_init = np.random.uniform(low=lb_init, high=ub_init)
            sol1 = solve_ivp(MAK_rsc, tspan1, x_init, args=(kappa1,), method='RK45')
            sol_mat1 = sol1.y.T
            length_tspan = sol_mat1.shape[0]
            final_val_mat[:, iter_init - 1] = sol_mat1[length_tspan - 1, :]

            if int(np.floor(0.8 * length_tspan)) >= 1:
                latter_val_mat[:, iter_init - 1] = sol_mat1[int(np.floor(0.8 * length_tspan)), :]

        upp_val = np.array([quantile(final_val_mat[i, :], 0.6) for i in range(num_S)])
        low_val = np.array([quantile(final_val_mat[i, :], 0.4) for i in range(num_S)])

        for i in range(num_S):
            if upp_val[i] - low_val[i] < eps_acr and low_val[i] > thres_positive:
                list_acr_id_par[i, iter_par - 1] = 1
            if np.mean(latter_val_mat[i, :]) > ub_init[i] and np.mean(final_val_mat[i, :]) > np.mean(latter_val_mat[i, :]) + 2:
                list_unbnd_id_par[i, iter_par - 1] = 1

    for i in range(num_S):
        if np.sum(list_acr_id_par[i, :]) > 0:
            list_acr_id[i, iter_network - 1] += 1
        if np.prod(list_acr_id_par[i, :]) > 0:
            list_acr_id[i, iter_network - 1] += 1
        if np.sum(list_unbnd_id_par[i, :]) > 0:
            list_unbnd_id[i, iter_network - 1] += 1
        if np.prod(list_unbnd_id_par[i, :]) > 0:
            list_unbnd_id[i, iter_network - 1] += 1

    matrix_R_id[:, iter_network - 1] = list_R_id

# Convert arrays to pandas DataFrames and save to CSV files
list_acr_id_df = pd.DataFrame(list_acr_id)
list_unbnd_id_df = pd.DataFrame(list_unbnd_id)
matrix_R_id_df = pd.DataFrame(matrix_R_id)

path = "/Users/hyukpyohong/Dropbox/CRN_dynamicACR/Codes/data/"
list_acr_id_df.to_csv(path + f"list_acr_id_S{num_S}R{num_R}_max_ord{max_order}.csv", header=False, index=False)
list_unbnd_id_df.to_csv(path + f"list_unbnd_id_S{num_S}R{num_R}_max_ord{max_order}.csv", header=False, index=False)
matrix_R_id_df.to_csv(path + f"matrix_R_id_S{num_S}R{num_R}_max_ord{max_order}.csv", header=False, index=False)
